# Base classes
> Base configuration classes and utilities for the logging library.

In [1]:
#| default_exp client.base

In [2]:
#| export
from dc_logger.client.Log import LogEntry, LogLevel

In [3]:
#| exporti

from typing import Optional, List, Dict, Any, Literal
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum



## Service Handler defines 'route functions' for how to interact with a service.

In [4]:
#| export


# Type for valid output modes
OutputMode = Literal["cloud", "console", "file", "multi"]


@dataclass
class ServiceConfig(ABC):
    """abstract base class for service-specific configuration settings"""

    output_mode: OutputMode

    def __post_init__(self):
        self.validate_config()


    @abstractmethod
    def validate_config(self) -> bool:
        """Validate the configuration"""
        raise NotImplementedError()

@dataclass
class Handler_BufferSettings(ABC):
    """abstract base configuration for logging configuration settings"""

    batch_size: int = 100
    flush_interval: int = 30  # seconds
    max_buffer_size: int = 1000

@dataclass
class ServiceHandler(ABC):
    """defines how a handler communicates with services to create logs"""

    buffer_settings: Handler_BufferSettings

    service_config: Optional[ServiceConfig] = None # has authentication and connection details to service1

    buffer : List[LogEntry] = field(default_factory = lambda: list )

    # @classmethod
    # def from_config(cls, service_config: ServiceConfig):
        
    #     hc = cls(
    #         service_config = service_config
            
    #     )
        
    #     # if hasattr(config, 'to_platform_config') and callable(getattr(config, 'to_platform_config')):
    #     #     hc.platform_config = config.to_platform_config()
    #     return hc
    
    def validate_config(self) -> bool:
        if not self.service_config:
            raise ValueError("Service configuration is not set.")
        
        return self.service_config.validate_config()

    @abstractmethod
    async def write(self, entries: List[LogEntry]) -> bool:
        """Write log entries to destination"""
        pass

    @abstractmethod
    async def flush(self) -> bool:
        """Flush any buffered entries"""
        pass

    async def close(self):
        """Clean up resources"""
        pass


In [5]:
# | export

@dataclass
class HandlerInstance:
    """Wraps a ServiceHandler with filtering logic (log level and method filtering)"""
    
    service_handler: ServiceHandler

    handler_name: str = None # friendly name for the handler

    log_level: LogLevel = LogLevel.INFO  # minimum log level to log.

    log_method: List[str] = field(
        default_factory=lambda: ["POST", "PUT", "DELETE", "PATCH", "COMMENT"]
    )
    # filtered list of API requests to log, generally won't log GET requests

    def __post_init__(self):
        if not self.handler_name:
            self.handler_name = f"{self.service_handler.__class__.__name__}"
        self.validate_config()

    def validate_config(self) -> bool:
        """Validate the configuration"""
        if not self.service_handler:
            raise ValueError("must set a service handler")
        
        return self.service_handler.validate_config()
    
    async def write(self, entry: LogEntry) -> bool:
        """Write log entry to destination with filtering"""
        # Filter by log level
        if entry.level.value < self.log_level.value:
            return False
            
        # Filter by log method
        if entry.method and entry.method not in self.log_method:
            return False
        
        # Delegate to service handler
        await self.service_handler.write([entry])
        return True

    async def flush(self) -> bool:
        """Flush any buffered entries"""
        return await self.service_handler.flush()

    async def close(self):
        """Clean up resources"""
        await self.service_handler.close()

In [6]:
#| export

from dc_logger.client.Log import CorrelationManager

@dataclass
class Logger:
    """Enhanced logger with structured logging and automatic correlation tracking"""
    
    handlers: List[HandlerInstance] = field(default_factory=list)
    app_name: Optional[str] = "default_app"
    min_level: LogLevel = LogLevel.INFO  # Minimum level to log
    
    # Correlation manager (auto-initialized)
    correlation_manager: Optional[CorrelationManager] = field(default_factory=CorrelationManager)
    
    def __post_init__(self):
        """Initialize correlation manager"""
        if self.correlation_manager is None:
            self.correlation_manager = CorrelationManager()
    
    async def log(self, level: LogLevel, message: str, **context) -> bool:
        """Core logging method - creates entry with auto-correlation and writes to handlers"""
        
        # Check if we should log this level
        if level.value < self.min_level.value:
            return True
        
        # Auto-generate or get existing correlation
        if self.correlation_manager and 'correlation' not in context:
            correlation = self.correlation_manager.get_or_create_correlation()
            context['correlation'] = correlation
        
        # Create log entry
        entry = LogEntry.create(
            level=level,
            message=message,
            app_name=self.app_name,
            user=context.get("user"),
            action=context.get("action"),
            entity=context.get("entity"),
            status=context.get("status", "info"),
            duration_ms=context.get("duration_ms"),
            correlation=context.get("correlation"),
            multi_tenant=context.get("multi_tenant"),
            http_details=context.get("http_details"),
            extra=context.get("extra", {}),
        )
        
        # Write to all handlers (handlers manage their own buffering)
        for handler in self.handlers:
            await handler.write(entry)
        
        return True
    
    async def write(self, entry: LogEntry):
        """Direct write - for compatibility"""
        for handler in self.handlers:
            await handler.write(entry)
    
    # Convenience methods for different log levels
    async def debug(self, message: str, **context) -> bool:
        """Log DEBUG level message"""
        return await self.log(LogLevel.DEBUG, message, **context)
    
    async def info(self, message: str, **context) -> bool:
        """Log INFO level message"""
        return await self.log(LogLevel.INFO, message, **context)
    
    async def warning(self, message: str, **context) -> bool:
        """Log WARNING level message"""
        return await self.log(LogLevel.WARNING, message, **context)
    
    async def error(self, message: str, **context) -> bool:
        """Log ERROR level message"""
        return await self.log(LogLevel.ERROR, message, **context)

    async def critical(self, message: str, **context) -> bool:
        """Log CRITICAL level message"""
        return await self.log(LogLevel.CRITICAL, message, **context)

    def create_entry(self, level: LogLevel, message: str, **kwargs) -> LogEntry:
        """Create a LogEntry without logging it (for manual control)"""
        # Auto-generate or get existing correlation
        if self.correlation_manager and 'correlation' not in kwargs:
            correlation = self.correlation_manager.get_or_create_correlation()
            kwargs['correlation'] = correlation
        
        entry = LogEntry.create(
            level=level,
            message=message,
            app_name=self.app_name,
            **kwargs
        )
        return entry
    
    def start_new_trace(self) -> str:
        """Start a completely new trace to group a bundle of related logs.
        
        Use this to separate different operations:
        - Bundle 1: User login flow
        - Bundle 2: Data processing
        - Bundle 3: Report generation
        
        Returns the new trace_id.
        """
        if not self.correlation_manager:
            self.correlation_manager = CorrelationManager()
        return self.correlation_manager.start_new_trace()
    
    def start_request(self, parent_trace_id: Optional[str] = None, auth=None, is_pagination_request: bool = False) -> str:
        """Start a new request context and return request ID"""
        if not self.correlation_manager:
            self.correlation_manager = CorrelationManager()
        return self.correlation_manager.start_request(parent_trace_id, auth, is_pagination_request)
    
    def end_request(self):
        """End the current request context (also clears trace)"""
        # Clear context variables for this request
        if self.correlation_manager:
            self.correlation_manager.trace_id_var.set(None)
            self.correlation_manager.request_id_var.set(None)
            self.correlation_manager.span_id_var.set(None)
            self.correlation_manager.correlation_var.set(None)
    
    async def close(self):
        """Clean up resources"""
        # Close all handlers
        for handler in self.handlers:
            await handler.close()
    
    # def get_cloud_config(self) -> Dict[str, Any]:
    #     return {"cloud_provider": "multi"}

    # def get_handler_configs(self) -> List[Dict[str, Any]]:
    #     return [
    #         {
    #             "type": handler.type,
    #             "config": handler.config,
    #             "cloud_config": (
    #                 handler.config.to_platform_config()
    #                 if handler.type == "cloud"
    #                 else None
    #             ),
    #         }
    #         for handler in self.handlers
    #     ]

    # @classmethod
    # def create(
    #     cls,
    #     handlers: List[Dict[str, Any]],
    #     level: LogLevel = LogLevel.INFO,
    #     batch_size: int = 100,
    #     flush_interval: int = 30,
    #     **kwargs
    # ) -> "MultiHandlerLogConfig":
    #     handler_configs = [
    #         HandlerConfig(type=h["type"], config=h["config"]) for h in handlers
    #     ]
    #     return cls(
    #         handlers=handler_configs,
    #         level=level,
    #         batch_size=batch_size,
    #         flush_interval=flush_interval,
    #         **kwargs
    #     )
    

In [7]:
#| hide
import nbdev; 
nbdev.nbdev_export('./base.ipynb')